In [1]:
import SigVarGen as svg
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
def generate_single_signal(domain):
    """
    Generate a single signal for a given domain, including base signal and interrupts.
    """
    num = random.uniform(0.3,0.7)
    drop=False
    signal_range, interrupt_range = svg.generate_device_parameters(svg.EMBEDDED_DEVICE_RANGES, drop=drop, frequency_follows_amplitude=False, split_ratio=num)

    amplitude_range = signal_range[domain]['amplitude']
    frequency_range = signal_range[domain]['frequency']

    if isinstance(frequency_range, dict):
        temp = random.choice(list(frequency_range.keys()))
        frequency_range = frequency_range[temp]
    else:
        temp = 0

    t = np.linspace(0, 1, 1000)
    n_sinusoids = random.randint(50, 100)

    disperse = random.choice([True, False])
    complex_iter = random.randint(0,3)

    base_wave, sinusoids_params = svg.generate_signal(t, n_sinusoids, amplitude_range, frequency_range)

    wave, interrupt_params = svg.add_interrupt_with_params(t, base_wave.copy(), domain, signal_range, interrupt_range, 
                                            temp, disperse=disperse, drop=drop, complex_iter=complex_iter)

    #plt.figure(figsize=(50, 5))
    #plt.plot(range(len(wave)), wave, label='wave')
    #plt.legend(fontsize=18)
    #plt.show()

    return t, wave, signal_range, interrupt_range, n_sinusoids, interrupt_params, temp

In [8]:
# Main data generation loop
data_list = []
num_variations = 10  # Number of variations for each signal
color = 'white'

for j in range(100):

    domain = random.choice(list(svg.EMBEDDED_DEVICE_RANGES.keys()))
    _, wave, signal_range, interrupt_range, _, _, _ = generate_single_signal(domain)

    noise_power, _ = svg.generate_noise_power(wave, snr_range=(-10, 20))

    data = {
        'domain': domain,
        'Stationarity': "Non-Stationary Power",
        'color' : color,
        'noise_power_σ^2': noise_power,
        'signal_range': signal_range[domain],
        'response_range': interrupt_range[domain]
    }

    data['num'] = j
    data['wave'] = wave

    mod_envelope = random.choice(svg.noise_funcs)

    if mod_envelope is not None:
        mod_envelope['param'][0] = random.uniform(mod_envelope['param'][0], mod_envelope['param'][1])
        mod_envelope['param'][1] = random.uniform(mod_envelope['param'][0], mod_envelope['param'][1])
        data['mod_envelope'] = mod_envelope['func'].__name__
    else:
        data['mod_envelope'] = None

    #print(color, domain, mod_envelope)

    for nv in range(num_variations):
        # Generate variations
        variation_wave = wave.copy()

        # Add noise
        noisy_wave = svg.add_colored_noise(variation_wave, noise_power, npw=[1,1], mf=[1,1], color=color, mod_envelope=mod_envelope)

        data[f'noisy_wave_{nv}'] = noisy_wave

        #plt.figure(figsize=(50, 5))
        #plt.plot(range(len(noisy_wave)), noisy_wave, label='noise', linewidth=1)
        #plt.plot(range(len(wave)), wave, label='wave', linewidth=5)
        #plt.title(f"noisy_wave", fontsize=22)
        #plt.legend(fontsize=18)
        #plt.show()

    data_list.append(data)

df = pd.DataFrame(data_list)

(483, 585)
(710, 767)
(789, 879)
(385, 449)
(122, 186)
(16, 106)
(405, 438)
(888, 921)
(499, 595)
(774, 804)
(483, 575)
(146, 216)
(793, 899)
(729, 743)
(207, 221)
(62, 170)
(508, 600)
(713, 805)
(799, 897)
(70, 96)
(143, 169)
(497, 595)
(286, 288)
(874, 876)
(588, 670)
(177, 230)
(737, 801)
(74, 101)
(889, 971)
(678, 737)
(121, 180)
(573, 674)
(422, 450)
(326, 354)
(229, 329)
(793, 854)
(335, 375)
(60, 132)
(575, 632)
(895, 952)
(126, 238)
(318, 407)
(34, 67)
(946, 979)
(362, 447)
(850, 882)
(29, 133)
(494, 572)
(323, 426)
(717, 779)
(464, 526)
(71, 182)
(919, 987)
(512, 555)
(466, 543)
(457, 549)
(633, 697)
(215, 279)
(894, 989)
(66, 82)
(462, 478)
(775, 852)
(662, 725)
(12, 75)
(110, 188)
(636, 637)
(207, 309)
(507, 593)
(380, 453)
(459, 506)
(241, 288)
(739, 842)
(191, 202)
(897, 993)
(656, 717)
(271, 350)
(802, 832)
(553, 583)
(71, 182)
(776, 794)
(347, 365)
(41, 152)
(230, 286)
(135, 214)
(368, 422)
(698, 752)
(287, 360)
(859, 914)
(98, 161)
(299, 334)
(549, 584)
(761, 840)
(923,

In [9]:
df

,domain,Stationarity,color,noise_power_σ^2,signal_range,response_range,num,wave,mod_envelope,noisy_wave_0,noisy_wave_1,noisy_wave_2,noisy_wave_3,noisy_wave_4,noisy_wave_5,noisy_wave_6,noisy_wave_7,noisy_wave_8,noisy_wave_9
0,Smart Watches,Non-Stationary Power,white,1.212785e-09,"{'amplitude': (0, 0.0003550244742431865), 'fre...","{'amplitude': (0.0003550244742431865, 0.0009),...",0,"[0.00018924852329410555, 0.0001619073068159949...",envelope_random_walk,"[0.00018927087235781417, 0.0001619073068159949...","[0.00018925520375733052, 0.0001619073068159949...","[0.0001892560001471464, 0.0001619359122375728,...","[0.00018925193423104586, 0.0001619073068159949...","[0.0001892593236935733, 0.0001619073068159949,...","[0.00018925096584903215, 0.0001618629188389211...","[0.0001892478259569292, 0.0001619073068159949,...","[0.00018923038212551453, 0.0001619073068159949...","[0.00018924755889014212, 0.0001618894194757697...","[0.00018922660914213888, 0.0001619073068159949..."
1,Smartphones,Non-Stationary Power,white,3.758183e-02,"{'amplitude': (0, 0.46882087075336304), 'frequ...","{'amplitude': (0.46882087075336304, 1), 'frequ...",1,"[0.24602964692528917, 0.18865151615869855, 0.2...",envelope_linear,"[0.26501193427753117, 0.15927303419710215, 0.2...","[0.05647721998185623, 0.0586899043843217, 0.19...","[0.2542122725806157, 0.2354783000500415, 0.129...","[0.17458639675982854, 0.40638470727230075, 0.1...","[0.43576982328901337, 0.25615332989615885, 0.1...","[0.17711334127407957, 0.1281846803538884, 0.14...","[0.1891176628726659, 0.05497013259377276, 0.08...","[0.4440060211282119, 0.15882076737653944, 0.22...","[0.29209712052537445, 0.32546458737966405, 0.2...","[0.0700458152594366, 0.00540814136055362, 0.03..."
2,Automotive Sensors,Non-Stationary Power,white,1.216579e-06,"{'amplitude': (0, 0.003815986957399251), 'freq...","{'amplitude': (0.003815986957399251, 0.01), 'f...",2,"[0.0017457426452310652, 0.0015548320214320553,...",None,"[0.0018368283548911082, 0.0010309657779342398,...","[0.00017830315759898778, 0.0019094128798032117...","[0.002167870406948902, 0.0006993301700539155, ...","[0.0028690556773982227, 0.0028022796783906474,...","[0.001062948786077475, 0.0031425479387494192, ...","[0.0041592547797221265, 0.0033944020033535712,...","[0.001628924366019415, 0.0014139566961289978, ...","[0.0019637770904374894, 7.972225838704045e-06,...","[0.000444751816721311, 0.0009586100620239108, ...","[0.001666524854888323, 0.0009541271264289305, ..."
3,Home Automation Devices,Non-Stationary Power,white,2.826062e-10,"{'amplitude': (0, 0.0006062221645131147), 'fre...","{'amplitude': (0.0006062221645131147, 0.0009),...",3,"[0.0001637411984939587, 0.0003515983854259461,...",envelope_blockwise,"[0.00016373869565553748, 0.0003515942799779332...","[0.0001637410830906432, 0.0003515998252405588,...","[0.00016373577189812027, 0.0003515812163148780...","[0.0001637400464367478, 0.00035160327618157395...","[0.0001637390841255887, 0.0003515999738630392,...","[0.0001637400322206041, 0.00035160450307840455...","[0.00016374074922034744, 0.0003515995979657551...","[0.00016372903992292127, 0.0003515951179237540...","[0.00016374414535845646, 0.0003515980647888275...","[0.00016374167514451271, 0.0003515993195458251..."
4,Smart Watches,Non-Stationary Power,white,2.123565e-09,"{'amplitude': (0, 0.0002829566803609796), 'fre...","{'amplitude': (0.0002829566803609796, 0.0009),...",4,"[0.00017001651675531773, 6.811840331588334e-05...",envelope_linear,"[0.00017004536573816124, 6.812669445995825e-05...","[0.00017004161780975573, 6.809611274559135e-05...","[0.0001699932916695958, 6.806667990653946e-05,...","[0.00017005482254338724, 6.814073825918337e-05...","[0.00017006198958320704, 6.810869559887953e-05...","[0.000170057963096775, 6.806481942916518e-05, ...","[0.00017005404100093027, 6.810372741596395e-05...","[0.00016997169084239348, 6.807974215262398e-05...","[0.00016998297363267153, 6.811003709055179e-05...","[0.00017005422896579893, 6.809841785019722e-05..."
...,...,...,...,.

In [10]:
df.to_pickle(r'../datasets/ESdevices_nonstatpower_whitenoise_short.pkl')